In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv

In [2]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [3]:
#definisco una matrice k x (n-k)
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [10]:
#mi definisco una variabile per il numero di colonne ridotto che voglio considerare
colonne_max = 12

In [11]:
#FACCIO METODO DI MONTECARLO
#numero iterazioni Monte Carlo
num_iterazioni = 1000  

def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

#bette but cannot count operations
def ordina_matrice_ottimizzata(matrice, colonne_max):
    print("Entro in ordina matrice ottimizzata")
    
    start = time.perf_counter()
    
    n = matrice.shape[0]
    n_colonne = matrice.shape[1]
    
    # Array indici originali
    indici = np.arange(n)
    indici1 = np.arange(n)

    #start = time.perf_counter()
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice1 = ordinato
    end = time.perf_counter() #these 2 for make file csv only count/ord
    tempo = (end - start) * 1000
    indici = np.lexsort([matrice1[:, i] for i in range(n_colonne - 1, -1, -1)]) 
    
    #check duplicate rows 
    if has_duplicate_rows(matrice1[indici]):
        tempo = (end - start) * 1000
        print("Algoritmo ha almeno due righe uguali --> fallisco")
        return False, [], 0, 0, tempo, 0.0
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #tempo = (end - start) * 1000
    
    #sorting with limited columns
    start = time.perf_counter()
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice2 = ordinato
    end = time.perf_counter() #these 2 for make file csv only count/ord
    tempo1 = (end - start) * 1000
    #lexsort implemnets a timesort algorithm thet is better than a quick sort in worst case
    indici1 = np.lexsort([matrice2[:, i] for i in range(colonne_max - 1, -1, -1)])
    #check equality between two index
    if has_duplicate_rows(matrice2[indici1]):
        tempo1 = (end - start) * 1000
        print("Algoritmo ha almeno due righe uguali --> fallisco")
        return False, [], 0, 0, tempo, tempo1
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #tempo1 = (end - start) * 1000
    if np.array_equal(matrice1[indici], matrice2[indici1]):
        #se le due matrici ottenute dai due ordinamenti sono uguali allora torno true, indici, numero di confronti e tempi
        #print(matrice1[indici], matrice2[indici1])
        return True, indici, 0, 0, tempo, tempo1
    else:
        print("Non sono uguali")
        return False, [], 0, 0, tempo, tempo1

   

#fa il procedimento di ordinamento tra le righe (che pero sono già ordinate al loro interno)
def ordina_matrice(matrice):
    print("Entro in ordina matrice")
    #righe matrice
    n = matrice.shape[0]  
    #contatore confronti
    confronti = 0  
    confronti1 = 0
    #array indici originali
    indici = np.arange(n)  
    indici1 = np.arange(n)
    #print(matrice)
    tempo = 0.0
    tempo1 = 0.0

    #l'ho messo per provare a vedere differenze nel solo caso in cui il primo ordinamento non vada in "errore" (due righe uguali)
    #tempo_effettivo = 0.0
    #l'ho messo per provare a vedere differenze nel solo caso in cui il primo ordinamento non vada in "errore" (due righe uguali)
    #tempo1_effettivo = 0.0

    #voglio vedere il tempo di ogni esecuzione 
    start = time.perf_counter()
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice1 = ordinato
    #la variabile esci non mi serve più
    esci = False
    #bubble sort per conteggio confronti
    for i in range(n):
        for j in range(i + 1, n):
            #confronto per primo el riga poi secondo e cosi via
            for k in range(matrice1.shape[1]):  
                confronti += 1
                #sono ordinate tra loro, vado avanti
                if matrice1[indici[i], k] < matrice1[indici[j], k]: 
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice1[indici[i], k] > matrice1[indici[j], k]:
                    #sposto indice sopra
                    indici[i], indici[j] = indici[j], indici[i]
                    #print("Indici dopo lo spostamento:", indici)
                    break 
                if k == (n_k - 1):
                    end = time.perf_counter()
                    esci = True
                    tempo = (end-start)*1000
                    print("Algoritmo ha almeno due righe uguali --> fallisco")
                    return False, [], confronti, confronti1,tempo, 0.0
            if esci:
                break
        if esci:
            break

    end = time.perf_counter()  
    #finisce l'esecuzione con tutte le colonne      
    tempo = (end-start)*1000

    #voglio vedere durata esecuzione con meno colonne
    start = time.perf_counter()
    #la variabile esci non mi serve più
    ordinato = []
    for riga in matrice:
        ordinato.append(np.sort(riga))
    
    ordinato = np.array(ordinato)
    matrice2 = ordinato
    esci = False
    for i in range(n):
        for j in range(i + 1, n):
            #confronto per primo el riga poi secondo e cosi via
            for k in range(colonne_max):  
                confronti1 += 1
                #sono ordinate tra loro, vado avanti
                if matrice2[indici1[i], k] < matrice2[indici1[j], k]: 
                    break
                #devo mettere la riga sopra quella con cui è confrontata e andare avanti
                elif matrice2[indici1[i], k] > matrice2[indici1[j], k]:
                    #sposto indice sopra
                    indici1[i], indici1[j] = indici1[j], indici1[i]
                    #print("Indici dopo lo spostamento:", indici)
                    break 
                    #se uguali continua a confrontare la prossima colonna
                if k == (colonne_max - 1):
                    esci = True
                    end = time.perf_counter()
                    tempo1 = (end-start)*1000
                    print("Algoritmo ha almeno due righe (CORTE) uguali --> fallisco")
                    return False, [], confronti, confronti1, tempo, tempo1
            if esci:
                break
        if esci:
            break        

    #print(matrice[indici])
    #print(matrice[indici1])
    end = time.perf_counter()
    #finisce confronto con colonne ridotte
    tempo1 = (end-start)*1000
    #print("Confronti = ", confronti, "Confronti 1 = ", confronti1)
    if np.array_equal(matrice1[indici], matrice2[indici1]):
        #se le due matrici ottenute dai due ordinamenti sono uguali allora torno true, indici, numero di confronti e tempi
        return True, indici, confronti, confronti1, tempo, tempo1
    else:
        return False, [], confronti, confronti1, tempo, tempo1
    
# Funzione per ordinare le righe con contatori di swap. In num_confronti metto confronti in # sufficienti metto confronti1. 
def ordina_per_riga_con_contatori(matrice, num_swap_righe):
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    print("Funzione ordina_per_riga")
    posizioni_originali = np.arange(matrice.shape[0])
        
    ris_confronto, indici_ordinamento, num_confronti,conta_sufficienti, tempo, tempo1 = ordina_matrice_ottimizzata(matrice, colonne_max)

    #se torno true quindi due matrici sono uguali
    if ris_confronto:
        # Conta gli swap solo quando la posizione delle righe cambia
        for i, indice in enumerate(indici_ordinamento):
            if indice != posizioni_originali[i]:
                num_swap_righe += 1
        return matrice[indici_ordinamento], num_swap_righe, num_confronti, conta_sufficienti, tempo, tempo1
    else:
        return [], num_swap_righe, num_confronti, conta_sufficienti, tempo, tempo1
    


#POTREI DOVER IMPLEMENTARE UNA COSA SIMILE A SOPRA PER IL COUNT DEI CONFRONTI (LEXSORT NON CE L'HA)
#ordinare le colonne con contatori di ordinamenti e swap
def ordina_per_colonna_con_contatori(matrice, righe, num_swap_colonne):
    print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    posizioni_originali = np.arange(matrice.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    indici_ordinamento = np.lexsort([matrice[rig, :] for rig in righe[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, indice in enumerate(indici_ordinamento):
        if indice != posizioni_originali[i]:
            num_swap_colonne += 1
    
    #riordina la matrice in base agli indici trovati
    matrice_ordinata = matrice[:, indici_ordinamento]
    return matrice_ordinata, num_swap_colonne

def test_algoritmo(matrice1, num_swap_righe, num_swap_colonne):
    print("Funzione test uguaglianza")
    start_time = time.perf_counter()
    #intervallo corretto per le righe
    righe = range(k)      
    
    print("Faccio ordina per riga con contatori")
    ordinata1, num_swap_righe, num_confronti, conta_sufficienti, tempo, tempo1 = ordina_per_riga_con_contatori(matrice1, 
                                                                num_swap_righe)
    print("Ordino le colonne")
    #se ordinata è praticamente vuoto vuol dire che ordina per riga per contatori non ha passato qualche test
    if len(ordinata1) == 0:
        end_time = time.perf_counter()
        durata = (end_time - start_time)*1000
        return False, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata, tempo, tempo1
    

    cf1, num_swap_colonne = ordina_per_colonna_con_contatori(ordinata1, righe, num_swap_colonne)
    end_time = time.perf_counter()
    durata = (end_time - start_time)*1000
    return True, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata, tempo, tempo1

#Funzione per eseguire la simulazione di Monte Carlo
def simulazione_montecarlo(num_iterazioni):
    #contatore simulazione accettabile
    simulazione_accettabile = 0
    #contatore swap righe
    num_swap_righe = 0  
    #contatore swap colonne
    num_swap_colonne = 0  
    #contatore confronti righe, dove c'è sufficienti è perchè si riferisce a test con meno colonne
    confronti_per_test = []
    sufficienti_per_test = []
    vector_time = []
    vector_time1 = []
    num_confronti_accumulatore = 0
    conta_sufficienti_accumulatore = 0
    
    #inizializzo la durata
    tempo_esecuzione = 0.0
    durata = 0
    num_confronti = 0
    conta_sufficienti = 0
    accumula_durata_tutte_colonne = 0.0
    accumula_durata_meno_colonne = 0.0

    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_seconda_revisione/dati_ottimizzato.csv'
    file_exists = os.path.exists(csv_file_path)
    with open(csv_file_path, mode='a', newline='') as file:
        writer_csv = csv.writer(file)

        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns'])

        for z in range(num_iterazioni):
            print(f"ITERAZIONE {z}")
            A = field(np.random.randint(0, field.order, size=(k, n_k)))
            while(has_duplicate_rows(A)):
               A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            
            esito, num_swap_righe, num_swap_colonne, num_confronti, conta_sufficienti, durata, tempo, tempo1 = test_algoritmo(A, num_swap_righe, num_swap_colonne)
            confronti_per_test = np.append(confronti_per_test,num_confronti)
            sufficienti_per_test = np.append(sufficienti_per_test,conta_sufficienti)
            num_confronti_accumulatore += num_confronti
            conta_sufficienti_accumulatore += conta_sufficienti

            #calcolo il tempo di ogni esecuzione, pure quelle negative
            tempo_esecuzione += durata
            #conto solo quelle che hanno successo anche con meno colonne
            writer_csv.writerow([3, z, colonne_max, tempo, tempo1, num_confronti, conta_sufficienti])

            if esito:
                simulazione_accettabile += 1
                vector_time.append(tempo)
                vector_time1.append(tempo1)
                accumula_durata_tutte_colonne += tempo
                accumula_durata_meno_colonne += tempo1
        
    #H0 è che la differenza tra numero dei confronti con tutte le colonne e quella con le colonne limitate NON è significativa
    t_stat, p_value = stats.ttest_rel(confronti_per_test, sufficienti_per_test)
    print(f"T-statistica: {t_stat}, P-value: {p_value}")
    alpha = 0.05
    if p_value < alpha:
        print("La differenza tra confronti normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")
    
    t_stat1, p_value1 = stats.ttest_rel(vector_time, vector_time1)
    print(f"T-statistica: {t_stat1}, P-value: {p_value1}")
    alpha = 0.05
    if p_value1 < alpha:
        print("La differenza tra tempi di esecuzione con confronti normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")
    #Faccio num_confronti_accumulatore/num_iterazioni, conta_sufficienti_accumulatore/num_iterazioni perchè in ogni caso
    #sia che la simulazione sia accettabile (ordinamento abbreviato = quello intero) che non, comunque conto il numero di
    #swap. Effettivamente potrei provare a fare un contatore del tempo solo per i casi positivi
    return tempo_esecuzione/num_iterazioni, num_swap_righe/num_iterazioni, num_swap_colonne/num_iterazioni, num_confronti_accumulatore/num_iterazioni, conta_sufficienti_accumulatore/num_iterazioni, (simulazione_accettabile/num_iterazioni)*100, (tempo/simulazione_accettabile), (tempo1/simulazione_accettabile), p_value, p_value1, confronti_per_test, sufficienti_per_test
    


In [12]:
# Output statistiche
# Esegui la simulazione
tempo_es_medio, num_medio_sw_ri, num_medio_sw_co, num_medio_confronti, num_medio_conf_sufficienti, accettati_perc, durata_ordine_completo, durata_ordine_meno_colonne, p_value, p_value1, confronti_per_test, sufficienti_per_test = simulazione_montecarlo(num_iterazioni)

ITERAZIONE 0
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 1
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 2
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 3
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 4
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 5
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione 

Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 17
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 18
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 19
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 20
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 21
Funzione test uguaglianza
Faccio ordina per riga con contatori
Funzione ordina_per_riga
Entro in ordina matrice ottimizzata
Ordino le colonne
Funzione ordina per colonna
ITERAZIONE 22
Funzione test uguagl

In [13]:
print()
print(f"Tempo esecuzione medio: {tempo_es_medio} ms")
print(f"Numero medio di swap righe: {num_medio_sw_ri}")
print(f"Numero medio di swap colonne: {num_medio_sw_co}")
print(f"Numero medio di confronti tra le righe in una matrice: {num_medio_confronti}")
print(f"Numero medio di confronti con meno colonne: {num_medio_conf_sufficienti}")
print(f"Percentuale di accettati (non controllo tutte le righe, mi fermo alle {colonne_max} prime colonne): {accettati_perc}%")
print(f"Durata media dell'ordinamento su base righe con tutte le colonne: {durata_ordine_completo}ms")
print(f"Durata media dell'ordinamento su base righe con meno colonne: {(durata_ordine_meno_colonne)}ms")


Tempo esecuzione medio: 12.95758224102974 ms
Numero medio di swap righe: 124.116
Numero medio di swap colonne: 124.105
Numero medio di confronti tra le righe in una matrice: 0.0
Numero medio di confronti con meno colonne: 0.0
Percentuale di accettati (non controllo tutte le righe, mi fermo alle 12 prime colonne): 99.3%
Durata media dell'ordinamento su base righe con tutte le colonne: 0.0019151027190270633ms
Durata media dell'ordinamento su base righe con meno colonne: 0.0010081722052173604ms


In [14]:
nome_file = "stats.txt"

with open(nome_file, 'a') as file:
    file.write(f"Qui c'è il limite colonne a: {colonne_max} (con metodo ottimizzato)\n")
    file.write(f"Tempo esecuzione medio: {tempo_es_medio} ms\n")
    file.write(f"Numero medio di swap righe: {num_medio_sw_ri}\n")
    file.write(f"Numero medio di swap colonne: {num_medio_sw_co}\n")
    file.write(f"Numero medio di confronti tra le righe in una matrice: {num_medio_confronti}\n")
    file.write(f"Numero medio di confronti con meno colonne: {num_medio_conf_sufficienti}\n")
    file.write(f"Percentuale di accettati (non controllo tutte le righe, mi fermo alle {colonne_max} prime colonne): {accettati_perc}%\n") 
    file.write(f"Durata media dell'ordinamento su base righe con tutte le colonne: {durata_ordine_completo} ms\n")  
    file.write(f"Durata media dell'ordinamento su base righe con meno colonne: {(durata_ordine_meno_colonne)} ms\n")  
    file.write(f"Per il t-test tra confronti (interi) e confronti con solo {colonne_max} colonne, p-value = {p_value}\n\n")
